<a href="https://colab.research.google.com/github/R0N3ldrt/Thesis/blob/main/new_training_CNN_v7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traininig CNN Spectrum

# Loading Necesary Libraries

In [4]:
# Importing necesary libraries
# Libraries for correct code execution 

import os, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
import random
import csv
import re
import array
import seaborn as sns
from tqdm import tqdm

from functools import reduce
from random import random, gauss
from math import modf, pi, cos, sin, sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from plotly.subplots import make_subplots
from scipy.signal import savgol_filter
from scipy.stats.stats import pearsonr
from scipy.stats import spearmanr

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from sklearn.mixture import GaussianMixture
import os, time, math, csv, joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture

import os, time, math, csv, joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings("ignore")

sns.set_theme()

# Load Enviroment

In [5]:
# BLOQUE PARA USAR DESDE COLAB

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

Mounted at /content/gdrive


In [6]:
# Lista para cambiar los paths rapido.
workers = ["Ronald", "Local"]

# Change the number to change the paths.
worker = workers[0]

if worker == "Ronald":
  path = "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

# Create training/testing df

In [7]:
def rand(start, end, num): # get random value function
    res = []
    for j in range(num):
        res.append(np.random.randint(start, end))
    return res

In [8]:
def calc_freq_behavior(working_df, selected_point, multi=False): # Individual freq calculator
  extra_data = working_df.iloc[:, 0:6]
  freq_selected = working_df.iloc[:, selected_point+6]

  freq_data_df = pd.merge(extra_data, freq_selected, left_index=True, right_index=True)
  freq_data_df['Distance_km'] = freq_data_df['Distance_km'].astype(int)

  headers = list(freq_data_df.columns.values)
  selected_val = round(float(headers[-1]), 5)

  distances = [x*80 for x in range(1, 26)]
  freq_behavior = {}
  f_behavior = {}

  for dist in distances: # loop all distances available

    distance_df = freq_data_df[(freq_data_df['Distance_km'] == dist)] # select all the row with current eval distance
    freq_distance_vals = distance_df[str(selected_val)].tolist()

    freq_mean = np.mean(freq_distance_vals)
    freq_std = np.std(freq_distance_vals)
    results = (freq_mean, freq_std)
    f_behavior[dist] = results

  freq_behavior[selected_val] = f_behavior
  return freq_behavior

In [9]:
def select_eval_cols(working_df, eval_cols_param): # retturn index of column to be evaluated
  if eval_cols_param[0] == 'even': 
    break_point = int((working_df.shape[1]-6)/(eval_cols_param[1]+1))
    selected_cols = []
    for col_idx in range(1, eval_cols_param[1]+1):
      selected_cols.append((col_idx*break_point)-1)
  else:
    selected_cols = rand(1, working_df.shape[1]-6, eval_cols_param[1])
    selected_cols = sorted(selected_cols)
  return selected_cols

In [10]:
# multi_params = (method, number_freq_to_analyze)
def multiple_freq_analyzer(working_df, multi_params): #method = 'rand' or 'even' (multiple freq behavior calculator)
  selected_cols = select_eval_cols(working_df, multi_params)

  multi_freq_behavior = {}
  for selected_point in tqdm(selected_cols):
    freq_behavior = calc_freq_behavior(working_df, selected_point)
    selected_v = [v for v in freq_behavior.keys()]

    selected_d = {}
    for data in freq_behavior.values():
      for k, v in data.items():
        selected_d[k] = v

    multi_freq_behavior[selected_v[0]] = selected_d
  return multi_freq_behavior

In [11]:
def freq_dict_to_df(multi_freq_behavior):
  distances = [x*80 for x in range(1, 26)]
  freq_mean_data = {'distances':distances}
  freq_std_data = {'distances':distances}

  for header, distances_dict in multi_freq_behavior.items():
    col_data_mean = []
    col_data_std = []
    for distance, results in distances_dict.items():
      col_data_mean.append(results[0])
      col_data_std.append(results[1])
    freq_mean_data[header] = col_data_mean
    freq_std_data[header] = col_data_std

  freq_mean_df = pd.DataFrame(freq_mean_data)
  freq_std_df = pd.DataFrame(freq_std_data)
  
  return freq_mean_df, freq_std_df

## Pearson Correletion

In [12]:
#eval_method = 'avg' or 'min' or 'max'

def pearson_matrix(working_df , source_sample_id):
  distances = [x*80 for x in range(1, 26)]
  result_data = {'source/target': distances}
  
  for target_dist in tqdm(distances): # loop all distances available
    min_result_target = []
    avg_result_target = []
    max_result_target = []
    for source_dist in distances: # loop all distances available
      source_df = working_df[(working_df['Distance_km'] == source_dist) & (working_df['PBRS_id'] == source_sample_id)]# select data of source (distance and sample_id) 
      source_data = source_df.iloc[0, 6:source_df.shape[1]].tolist()
      pearson_values = []
      for target_sample_id in working_df['PBRS_id'].unique(): # loop all samples id to then exclude the source sample id
        if target_sample_id != source_sample_id:
          target_df = working_df[(working_df['Distance_km'] == target_dist) & (working_df['PBRS_id'] == target_sample_id)] # select data of target
          target_data = target_df.iloc[0, 6:target_df.shape[1]].tolist()
          pearson_values.append(round(pearsonr(source_data, target_data)[0], 5)) # pearson correlation
      
      avg_result_target.append(np.mean(pearson_values))    
      max_result_target.append(np.max(pearson_values))
      min_result_target.append(np.min(pearson_values))
      
    result_data['min_'+str(target_dist)] = min_result_target
    result_data['avg_'+str(target_dist)] = avg_result_target
    result_data['max_'+str(target_dist)] = max_result_target

  pearson_matrix_df = pd.DataFrame.from_dict(result_data)
  #pearson_matrix_df = pearson_matrix_df.set_index('source/target')

  return pearson_matrix_df, result_data

## Mean/std analysis

In [13]:
def calc_freq_behavior(working_df, selected_point): # Individual freq calculator
  extra_data = working_df.iloc[:, 0:6]
  freq_selected = working_df.iloc[:, selected_point+6]

  freq_data_df = pd.merge(extra_data, freq_selected, left_index=True, right_index=True)
  freq_data_df['Distance_km'] = freq_data_df['Distance_km'].astype(int)

  headers = list(freq_data_df.columns.values)
  selected_val = round(float(headers[-1]), 5)

  distances = [x*80 for x in range(1, 26)]
  freq_behavior = {}
  f_behavior = {}

  for dist in distances: # loop all distances available

    distance_df = freq_data_df[(freq_data_df['Distance_km'] == dist)] # select all the row with current eval distance
    freq_distance_vals = distance_df[str(selected_val)].tolist()

    freq_mean = np.mean(freq_distance_vals)
    freq_std = np.std(freq_distance_vals)
    results = (freq_mean, freq_std)
    f_behavior[dist] = results
  freq_behavior[selected_val] = f_behavior
  return freq_behavior

In [14]:
def calc_freq_behavior_v2(working_df, amt_of_selected_point): # Individual freq calculator
  extra_data = working_df.iloc[:, 0:6]

  distances = [x*80 for x in range(1, 26)]

  std_combined = []
  for dist in distances: # loop all distances available
    std_all = []
    for val_idx in range(0, amt_of_selected_point):

      freq_selected = working_df.iloc[:, val_idx+6]
      freq_data_df = pd.merge(extra_data, freq_selected, left_index=True, right_index=True)
      freq_data_df['Distance_km'] = freq_data_df['Distance_km'].astype(int)

      headers = list(freq_data_df.columns.values)
      selected_val = round(float(headers[-1]), 5)
      
      distance_df = freq_data_df[(freq_data_df['Distance_km'] == dist)] # select all the row with current eval distance
      freq_distance_vals = distance_df[str(selected_val)].tolist()

      freq_std = np.std(freq_distance_vals)
      std_all.append(freq_std)
      std_avg = round(np.average(std_all), 5)
    std_combined.append(std_avg)


  data = {'source/target':distances, 'std':std_combined}
 
  # Create DataFrame
  std_df = pd.DataFrame(data)
  return std_df

In [18]:
# Get features 
def get_features(eval_freq_data_df, amt_sel_points):
  # get pearson values
  pearson_matrix_df, result_data = pearson_matrix(eval_freq_data_df, source_sample_id=1)

  # get std values
  std_df = calc_freq_behavior_v2(eval_freq_data_df, amt_of_selected_point = amt_sel_points)

  final_df = pd.merge(pearson_matrix_df, std_df, on = "source/target", how = "inner")
  return final_df

In [16]:
def train_test_split(working_df, amt_train = 0.8):
  print('Creating train/test split:')
  num_sample_train = math.ceil((working_df.shape[1]-6)*amt_train)
  num_sample_test = (working_df.shape[1]-6) - num_sample_train

  multi_params = ('rand', num_sample_train)  # (evenly separate points(other option 'rand'for randomly selected points), mount of points to be selected)
  multi_freq_behavior = multiple_freq_analyzer(working_df, multi_params)

  eval_points = select_eval_cols(working_df, multi_params) # get indexes of of points to be evaluated
  real_eval_idx = [eval_point+6 for eval_point in eval_points]

  extra_data = working_df.iloc[:, 0:6]
  eval_freq_df = working_df.iloc[:, real_eval_idx]
  train_df = pd.merge(extra_data, eval_freq_df, left_index=True, right_index=True) #df we only eval freqs TRAIN

  test_df= working_df.copy()
  test_df.drop(test_df.columns[real_eval_idx], axis=1, inplace=True) # df without eval freqs TEST
  return train_df, test_df

## Run train/test split

In [20]:
input_path = path + "/Spectrum/CNN/working_df.csv"

working_df = pd.read_csv(input_path)
working_df['Distance_km'] = working_df['Distance_km'].astype(int)

train_df, test_df = train_test_split(working_df, amt_train = 0.8)

Creating train/test split:


100%|██████████| 1654/1654 [00:43<00:00, 38.15it/s]


In [19]:

print('------------------------------')
# Get features 
print('Calculating features for train set:')
train_features_df = get_features(train_df, train_df.shape[1]-6)
print('------------------------------')
print('Calculating features for test set:')
test_features_df = get_features(test_df, test_df.shape[1]-6)

#final_final_df.to_csv(path+'/Spectrum/CNN/training_data_DNN.csv')

Creating train/test split:


100%|██████████| 1654/1654 [00:28<00:00, 57.93it/s]


------------------------------
Calculating features for train set:


100%|██████████| 25/25 [00:22<00:00,  1.11it/s]


------------------------------
Calculating features for test set:


100%|██████████| 25/25 [00:12<00:00,  2.05it/s]


In [ ]:
train_features_df.to_csv(path+'/Spectrum/CNN/training_data_DNN.csv')

In [ ]:
test_features_df.to_csv(path+'/Spectrum/CNN/testing_data_DNN.csv')

# Neural Network

## Pre-processing

In [ ]:
training_df=pd.read_csv(path+'/Spectrum/CNN/training_data_DNN.csv')

In [ ]:
testing_df=pd.read_csv(path+'/Spectrum/CNN/testing_data_DNN.csv')

In [ ]:
Y = training_df.iloc[:, 1].to_numpy().reshape(-1,1)
X = training_df.iloc[:, 2:training_df.shape[1]]

sc_input = MinMaxScaler()
sc_output = MinMaxScaler()
X_train = sc_input.fit_transform(X) # convert features to values from 0 to 1
Y_train = sc_output.fit_transform(Y) # convert distances to values from 0 to 1

## Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping


def ann():
    model = Sequential()
    model.add(Dense(76, input_dim = 76, activation = 'tanh'))
    model.add(Dense(38,activation='tanh'))
    model.add(Dense(19,activation='tanh'))
    model.add(Dense(1))
    model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
    return model


model_ann = KerasRegressor(build_fn=ann,epochs=5000,batch_size=32)

start_time = time.time()
callback = EarlyStopping(monitor='loss', patience=500)
model_ann.fit(X_train, Y_train, callbacks=[callback])
time_train_ann = time.time() - start_time

## Test Model

In [ ]:
testing_df=pd.read_csv(path+'/Spectrum/CNN/testing_data_DNN.csv')

Y = testing_df.iloc[:, 1].to_numpy().reshape(-1,1)
X = testing_df.iloc[:, 2:testing_df.shape[1]]


sc_input = MinMaxScaler()
sc_output = MinMaxScaler()
X_test = sc_input.fit_transform(X) # convert features to values from 0 to 1
Y_test = sc_output.fit_transform(Y) # convert distances to values from 0 to 1

In [ ]:
X_test = sc_input.transform(X)  #convert features two values between 0 and 1
Y_test = sc_output.transform(Y) #convert distances two values between 0 and 1

start_time = time.time()
Y_test_pred=model_ann.predict(X_test)

time_eval_ann=time.time()-start_time

real=list(list(zip(*Y_test))[0])

#real=list(map(list, zip(*Y_train)))
pred=list(Y_test_pred)
print(pred)


dist_min=sc_output.data_min_[0] # 80
dist_max=sc_output.data_max_[0] # 2000

#np.add = add values in a array
#np.multiply multiple values in a array
print(real)
real_abs=np.add(dist_min,np.multiply((dist_max-dist_min),real)) # convert back to distances values from scalar
real_abs=[int(np.round(i)) for i in real_abs]
#print(real_abs)
pred_abs=np.add(dist_min,np.multiply((dist_max-dist_min),pred)) # convert back to distances values from scalar
#print(pred_abs)

#np.divide = divide values in a array
error=np.divide(np.abs(np.subtract(np.array(real_abs),np.array(pred_abs))),np.array(real_abs))

res=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error})
display(res)
res.to_csv("results_pme_supervisedFeatures.csv", header=True, index=False)
plt.plot(real_abs,error,'bo')
plt.show()